# MLOps: Initial - Manual deployment & validation 

<div class="alert alert-warning"> 
	⚠️ <strong> PRE-REQUISITE: </strong> Before proceeding with this notebook, please ensure that you have executed the <code>1-data-prep-feature-store.ipynb</code> and <code>2-training-registry.ipynb</code> Notebooks</li>
</div>

## Contents

- [Introduction](#Introduction)
- [SageMaker Endpoint](#SageMaker-Endpoint)

## Introduction

This is our third notebook which will explore the model deployment of ML workflow.

Here, we will put on the hat of a `Data Scientist` or `ML Engineer` and will perform the task of model deployment which includes fetching the right model and deploying it for inference. This is often done during model development to validate consuming applications or inference code as well as for ad-hoc inference.  In the early stages of MLOps adoptions, this is often a manual task.  However, for models deployed that are serving live business critical production traffic - typically the deployment is handled through Infrastructure-as-Code(IaC)/Configuration-as-Code(CaC) outside the experimentation environment.  This allows for robust deployment patterns that support advanced deployment strategies (ex. A/B, Shadow, Blue/Green) as well as automatic rollback capabilities.

For this task we will be using Amazon SageMaker Model Hosting capabilities to manually deploy and perform inference.  In this  case, we will utilize a real-time endpoint; however, SageMaker has other deployment options to meet the needs of your use cases. Please refer to the provided [detailed guidance](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html) when choosing the right option for your use case. We'll also pull features for inference from the online SageMaker Feature Store.   Keep in mind, this is typically done through the consuming application and inference pipelines but we are showing it here to illustrate the manual steps often performed during validation stages of development. 

![Notebook3](./images/Notebook-3.png)

Let's get started!

**Imports**

In [ ]:
!pip install -U sagemaker

In [ ]:
%store -r

In [ ]:
from urllib.parse import urlparse
import io
import time
from sagemaker.model import ModelPackage
import boto3
import sagemaker
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer, CSVDeserializer
import numpy as np
import pathlib
from sagemaker.feature_store.feature_group import FeatureGroup
from helper_library import *

**Session variables**

In [ ]:
# Useful SageMaker variables
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role_arn= sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
s3_client = boto3.client('s3', region_name=region)
sagemaker_client = boto3.client('sagemaker')

## SageMaker Endpoint

You can deploy your trained model as [SageMaker hosted endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html) which serves real-time predictions from a trained model. The endpoint will retrieve the model created during training and deploy it within a SageMaker scikit-learn container. This all can be accomplished with one line of code. Note that it will take several minutes to deploy the model to a hosted endpoint.

Let's get the model we registered in the Model Registry.

In [ ]:
xgb_regressor_model = ModelPackage(
    role_arn,
    model_package_arn=model_package_arn,
    name=model_name
)

It's current status is `PendingApproval`. In order to use this model for offline predictions or as a real-time endpoint, we should adopt the practice of setting the status to `Approved` prior to deployment.   You can optionally register your model in `Approved` status at the time of model registration; however, this option allows for as secondary peer review approval workflow.  

In [ ]:
sagemaker_client.update_model_package(
    ModelPackageArn=xgb_regressor_model.model_package_arn,
    ModelApprovalStatus='Approved'
)

Now we can deploy it!

In [ ]:
from IPython.core.display import display, HTML
endpoint_name = f'{model_name}-endpoint-' + time.strftime('%Y-%m-%d-%H-%M-%S')
display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">The Endpoint</a> After About 5 Minutes</b>'.format(
            region, endpoint_name
        )
    )
)
xgb_regressor_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',  # 'ml.t2.medium',
    endpoint_name=endpoint_name
)

Let's test this real-time endpoint by passing it some data and getting a real-time prediction back.

## Use the test set to test the preditions

TODO: Fix - Pull data from the offline feature store for quick validation of model deployment

In [ ]:
import pandas as pd 
import os
import time
from glob import glob
# Copy the link of one of the csv files under ./data/processed/test/
path = './data/processed/test/'
csv_files = glob(os.path.join(path, "*.csv"))
df = pd.read_csv(csv_files[0])
df.head(5)


In [ ]:
# Attach to the SageMaker endpoint
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=CSVSerializer(),
    deserializer=CSVDeserializer()
)

dropped_df = df.drop(columns=["PRICE"])

# Get a real-time prediction (only predicting the 1st 5 row to reduce output size)
predictor.predict(dropped_df[:5].to_csv(index=False, header=False))

Predict using the raw number. Remember to apply the scaling to the input variable before prediction

In [ ]:
predictor.predict('-1.2786684709120923,-0.2238534355557023,-1.4253170501458523,-0.0020177318669674,-0.6703058406786521,1.330874688860325,-0.9766068758234135,1.0010005005003753')